In [25]:
import scipy as sp
import networkx as nx
import numpy as np
import math
import time

from networkx.utils import reverse_cuthill_mckee_ordering #RCM
from scipy.io import mmread
from scipy.sparse import csr_matrix

# DESCRIÇÃO - Função para receber matriz .mtx
# SAÍDA - MATRIZ
def load_matriz(name_mtz): 
    matriz = open(name_mtz, "r")
    return matriz

# DESCRIÇÃO - Funcao para retornar o maximo da largura de banda - é chamada antes e depois para calcular o valor da matriz inicial e final
# ENTRADA - Recebe como parâmetro uma matriz de adjacencia
# SAÍDA - Retorna o valor maior da largura de banda
def bandwidth_max(matadj): 
    vx = matadj.shape[0]
    vy = matadj.shape[1]
    maxim = 1
    for i in range(vx):
        for j in range(vy):
            if matadj[i,j] > 0 and matadj[i,j] > maxim:
                maxim = matadj[i,j]                
    return maxim 

# DESCRIÇÃO - verifica as arestas
# ENTRADA - Recebe como parâmetro uma matriz de adjacencia csr
# SAIDA Retorna matriz de adjacencia
def checks_edges(matadj_csr): 
    vx = matadj_csr.shape[0]
    vy = matadj_csr.shape[1]
    for i in range(vx):
        for j in range(vy):
            if (matadj_csr[i,j] != 0):
                if (j - i > 0):
                    matadj_csr[i,j] = j - i
                else:
                    matadj_csr[i,j] = -(j - i)
    return matadj_csr 

# DESCRIÇÃO - Função de menor grau utilizada para auxiliar na função do RCM

# ENTRADA - Recebe um grafo
# SAÍDA - Retorna o vertice de menor grau
def smallest_degree(G):
    return min(G, key=G.degree)


# DESCRIÇÃO - Função utiliza reverse_cuthill_mckee_ordering para executar a heuristica reverse_cuthill_mckee 
# no reverse ele Gera uma ordenação (permutação) dos nós do gráfico para fazer uma matriz esparsa.
# chama a função nx.to_scipy_sparse_matrix só que passando o grafo e com a ordenação gerada e retorna a matriz de adjacencia com as permutações feitas
# Retorna a matriz de adjacência do grafo como uma matriz esparsa.
# ENTRADA - Recebe um grafo
# SAÍDA - Retorna uma matriz esparsa atualizados depois de aplicar o rcm
def rcm(graph): 
    list_rcm = list(reverse_cuthill_mckee_ordering(graph,heuristic=smallest_degree))
    return checks_edges(nx.to_scipy_sparse_matrix(graph, nodelist=list_rcm))

# DESCRIÇÃO - Utiliza scipy_sparse_matrix para transformar a matriz em um grafo
# ENTRADA -  Matriz
# SAÍDA - GRAFO
def trans_graph(matriz):
    g = nx.from_scipy_sparse_matrix(matriz)
    return g

# DESCRIÇÃO - Função main, variavel matriz recebe a matriz adjacentes após realizar o calculo, logo depois
#transforma em grafo para a função rcm utiliza-lo e retornar a matriz esparsa.
# IMPRIME - A LARGURA DE BANDA INICIAL E FINAL
# IMPRIME - O TEMPO E A MATRIZ PERMUTADA
if __name__ == '__main__':
    name_mtz = ['mycielskian3.mtx','bcspwr02.mtx','iris_dataset_30NN.mtx','Trefethen_200.mtx','494_bus.mtx','662_bus.mtx']
    t_init_total = time.time()  
    for i_m in range(len(name_mtz)):
        print("MATRIZ - ", name_mtz[i_m])        
        representacao = 0
        count = 0
               
        print("Execução\t Largura de Banda Inicial \t Largura de Banda Final \t\t\t Tempo")
        while count <= 11:
            t_init = time.time()
            
            representacao=count+1
            matriz = checks_edges(sp.io.mmread(load_matriz(name_mtz[i_m])).tocsr()) 
            graph = trans_graph(matriz)
            matriz_reduced = rcm(graph)
            b_maxim = bandwidth_max(matriz)
            b_minim = bandwidth_max(matriz_reduced)
            print("%d \t\t\t\t %d \t\t\t\t %d  \t\t\t\t %f s\t\t\t\t " % (representacao, b_maxim, b_minim ,time.time() - t_init))
            
            matriz = []
            graph = []
            matriz_reduced = []
            b_maxim = []
            b_minim = []
            t_init = 0
            count += 1
        print("===========================================================================================================\n")
    print("Tempo total de todas as execuções:", time.time()-t_init_total, "s")

MATRIZ -  mycielskian3.mtx
Execução	 Largura de Banda Inicial 	 Largura de Banda Final 			 Tempo
1 				 3 				 2  				 0.015993 s				 
2 				 3 				 2  				 0.015999 s				 
3 				 3 				 2  				 0.008000 s				 
4 				 3 				 2  				 0.008000 s				 
5 				 3 				 2  				 0.008000 s				 
6 				 3 				 2  				 0.008077 s				 
7 				 3 				 2  				 0.000000 s				 
8 				 3 				 2  				 0.015625 s				 
9 				 3 				 2  				 0.000000 s				 
10 				 3 				 2  				 0.026537 s				 
11 				 3 				 2  				 0.008982 s				 
12 				 3 				 2  				 0.009007 s				 

MATRIZ -  bcspwr02.mtx
Execução	 Largura de Banda Inicial 	 Largura de Banda Final 			 Tempo
1 				 34 				 13  				 0.624372 s				 
2 				 34 				 13  				 0.528534 s				 
3 				 34 				 13  				 0.482654 s				 
4 				 34 				 13  				 0.468837 s				 
5 				 34 				 13  				 0.482527 s				 
6 				 34 				 13  				 0.518082 s				 
7 				 34 				 13  				 0.547526 s				 
8 				 34 				 13  				 0.571650 s				 
9 				 34 				 13  				 0.5092